# Анализ ответов на второй вопрос

Импортируем необходимые библиотеки

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import seaborn as sns
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import plotly.figure_factory as ff
import random
import math
from scipy.stats import shapiro
import scipy.stats as st
import stats_functions as sf
pd.options.mode.chained_assignment = None

Проводим операции по первоначальной очистке датафрейма, аналогичные описанным в первой части работы

In [ ]:
megafon_df = pd.read_csv('megafon.csv')
megafon_df.loc[megafon_df['Q1'] == '1, 3', 'Q1'] = '2'
megafon_df.loc[megafon_df['Q1'] == '5, 6', 'Q1'] = '5'
megafon_df.loc[megafon_df['Q1'] == 'Ужасно', 'Q1'] = '1'
megafon_df.loc[megafon_df['Q1'] == 'Поохое', 'Q1'] = '1'
megafon_df.loc[megafon_df['Q1'] == 'Очень  хорошо. Обслуживания  я довольно. Спасибо вам.555', 'Q1'] = '9'
megafon_df.loc[megafon_df['Q1'] == '3 тройка, связь отвратительная, жалко платить за такой тарив', 'Q1'] = '3'
megafon_df.loc[megafon_df['Q1'] == 'Немагу дать атценку денги незашто снимаеть скоро выклучаю', 'Q1'] = '1'
megafon_df.loc[megafon_df['Q1'] == '4. Тульская область Заокский район. Романовские дачи связи почти нет', 'Q1'] = '4'
megafon_df.loc[megafon_df['Q1'] == '10, 9', 'Q1'] = '10'
megafon_df.loc[megafon_df['Q1'] == 'Чем даль ше,тем лучше.Спасибо за ваш труд.Оценка 10 !', 'Q1'] = '10'
megafon_df.loc[megafon_df['Q1'] == 'ОЦЕНКА-3/НЕВАЖНО/', 'Q1'] = '3'
megafon_df.loc[megafon_df['Q1'] == 'Отвратительно', 'Q1'] = '1'
megafon_df.loc[megafon_df['Q1'] == '5, 7', 'Q1'] = '6'
megafon_df.loc[megafon_df['Q1'] == '5, 7', 'Q1'] = '6'
megafon_df.loc[megafon_df['Q1'] == '0', 'Q1'] = '1'
undefined_df = megafon_df[megafon_df['Q1'].isin(megafon_df['Q1'].value_counts().tail(25).index) | megafon_df['Q1'].isna()]
good_grade_df = megafon_df.loc[((megafon_df['Q1'] == '10') | (megafon_df['Q1'] == '9'))]

Так как ответ на второй вопрос должен последовать только если оценка меньше 9, на всякий случай проверим, что при оценках на первый вопрос 9 и 10 все ответы на второй вопрос отсутствуют.

In [ ]:
len(good_grade_df[~good_grade_df['Q2'].isna()])

Всё так и есть. Создадим датафрейм defined_df, который содержит только строки, где оценки на первый вопрос от 1 до 8 включительно, то есть подразумевающий наличие ответа на второй вопрос

In [ ]:
defined_df = megafon_df[~megafon_df['user_id'].isin(list(undefined_df['user_id']) + list(good_grade_df['user_id']))]

Посмотрим, сколько абонентов должны были ответить на второй вопрос и сколько абонентов не стали этого делать

In [ ]:
no_answer_df = defined_df[defined_df['Q2'].isna()]
print(len(defined_df), len(no_answer_df))

Как мы видим очень много абонентов не стали отвечать на второй вопрос. Почему? Варианта два:

&emsp;1. Есть вариант "Затрудняюсь ответить". Более аккуратные абоненты ответили так, а, например, менее аккуратные посчитали, что смысла отвечать так нет и просто не ответили никак.

&emsp;2. Есть вариант "Свой вариант". Возможно те, кто не ответили никак, просто посчитали за лишний труд расписывать свой вариант ответа.

Как бы то ни было оба этих варианта объединяет одно - такие абоненты не нашли среди предложенных вариантов ответа подходящий им. Логично объединить тех, кто ничего не ответил, и тех кто дал ответ  "Затрудняюсь ответить" или "Свой вариант". Давайте добавим тех, кто ничего не ответил к варианту "Затрудняюсь ответить".

Вообще говоря ответ "Свой вариант" тоже интересен для анализа, но так как нам не дали в исходных данных, что отвечали пользователи при своём варианте, то смело осуществим объединение, описанное выше. 

In [ ]:
defined_df = defined_df.fillna('6')

Посморим на состав ответов и их частоту

In [ ]:
defined_df['Q2'].value_counts()

Уберем из датафрейма строки с ответами на второй вопрос, которых не может быть. Их всего шесть, поэтому ничего страшного.

In [ ]:
defined_df = defined_df[~defined_df['Q2'].isin(['0, 3', '0, 1, 7', '1, 2, 34', '0, 05, 2, 27, 7', '10', '0'])]
defined_df = defined_df.reset_index()
defined_df.drop('index', axis = 1 , inplace = True)

&emsp;Так как абонент мог дать несколько ответов на второй вопрос, расширим наш датафрейм строками с отдельным ответом каждого абонента. То есть если абонент дал ответы 1, 2 и 3, то мы сделаем три строки с соответствующим одним ответом. В дальнейшем при статистическом анализе при рассмотрении например ответа абонента под номером 1 мы не будем учитывать строки с ответом этого же абонента на другие вопросы, чтобы не вносить погрешность, так как значения технических параметров для одного и того же абонента будут одинаковыми.

In [ ]:
for i, rating in enumerate(defined_df['Q2']):
    if len(rating) != 1:
        rating_list = rating.split(', ')
        defined_df.iloc[i, 2] = rating_list[0]
        for rat in rating_list[1:]:
            defined_df.loc[len(defined_df.index)] = list(defined_df.iloc[i, :])
            defined_df.iloc[len(defined_df.index) -  1, 2] = rat

Теперь мы можем привести столбцы с ответами на первый и второй вопрос к целому типу

In [ ]:
defined_df = defined_df.astype({"Q1": "Int64", "Q2": "Int64"})
defined_df

Получилось 3155 строк - вполне нормальный объем для анализа

Для удобства создадим словарик, учитывающий соответствие номера ответа и его текстового описания

In [ ]:
answers_dict = {1: 'Недозвоны,<br> обрывы<br> при<br> звонках',
                2: 'Время<br> ожидания<br> гудков<br> при<br> звонке',
                3: 'Плохое<br> качество<br> связи<br> в зданиях',
                4: 'Медленный<br> мобильный<br> Интернет',
                5: 'Медленная<br> загрузка<br> видео',
                6: 'Затрудняюсь<br> ответить',
                7: 'Свой<br> вариант'
               }

Давайте взглянем на распределение ответов на второй вопрос в нашем подредактированном датафрейме

In [ ]:
fig = px.histogram(defined_df, x = 'Q2', range_x = [0, 8], color = 'Q2')
fig.update_traces(marker_line_width=1,marker_line_color="white")
fig.update_layout(title_text="Распределение количества ответов абонентов на второй вопрос",
                  title_font=dict(size=22, color='blue'))
fig.update_xaxes(nticks=16)
fig.update_xaxes(title_font=dict(size=18, color='green'), title_text='Выбор абонента при ответе на второй вопрос')
fig.update_yaxes(title_font=dict(size=18, color='green'), title_text='Количество выбравших ответ')
for i, text in answers_dict.items():
    fig.add_annotation(x=i, y=100, text=text, showarrow=False, yshift=10)
fig.show()

Ну и то же самое в абсолютных числах

In [ ]:
defined_df['Q2'].value_counts()

Давайте посмотрим сначала на картину распределения технических параметров. (на диагонали графика)

In [ ]:
sns.pairplot(defined_df)

Аналогично первой части работы мы видим, что все тех.параметры связи кроме Total Traffic(MB) имеют ненормальное распределение с ярко выраженным правым "хвостом". Давайте подрежем "хвосты" по квантилю 0.97 и посмотрим на новую картину распределения.

In [ ]:
clean_df = defined_df.copy()
q_hi_list = [clean_df[col].quantile(0.97)  for col in clean_df.columns[4:]]
for i, col in enumerate(clean_df.columns[4:]):
    clean_df = clean_df[clean_df[col] < q_hi_list[i]]
clean_df.shape

In [ ]:
sns.pairplot(clean_df)

При этом, как видно, мы теряем не так много данных, зато распределения уже гораздо ближе к нормальным. Далее будем работать с таким датафреймом.

Перед статистическими тестами давайте ещё посмотрим на картину корелляции тех.параметров связи между собой

In [ ]:
corr = clean_df[clean_df.columns[3:]].corr()
sns.set(rc = {'figure.figsize':(15,8)})
sns.heatmap(corr,
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values,
            annot = True)

Аналогично первой части работы видно, что хорошо кореллируют между собой параметры, связанные с хорошим каналом передачи данных к абоненту. 

## Перейдём к статистическим тестам

Используем нашу универсальную функцию. Давайте же запустим нашу функцию сначала используя метод "by_func" (менее точный), а затем "prop" (более точный).

In [ ]:
res_df = sf.bootstrep(clean_df,
                      target_feature_idx = 2,
                      columns_idxs = [i for i in range(3, 11)],
                      hypotheses_list = [1, 1, 1, 0, 1, 0, 1, 0],
                      n = -1,
                      N_TRIAL = 3000,
                      func = np.mean,
                      method = 'by_func')

In [ ]:
s = res_df.style.apply(lambda x: ["background-color:lightgreen;" if i == 'Да' \
                              else "background-color:pink;" for i in x],
                   axis=1) ## Axis=1 : Row and Axis=0: Column

s

In [ ]:
res_df = sf.bootstrep(clean_df,
                      target_feature_idx = 2,
                      columns_idxs = [i for i in range(3, 11)],
                      hypotheses_list = [1, 1, 1, 0, 1, 0, 1, 0],
                      n = -1,
                      N_TRIAL = 3000,
                      func = np.mean,
                      method = 'prop')

In [ ]:
s = res_df.style.apply(lambda x: ["background-color:lightgreen;" if i == 'Да' \
                              else "background-color:pink;" for i in x],
                   axis=1) ## Axis=1 : Row and Axis=0: Column

s

А теперь глазами оцениваем все результаты и делаем выводы по оценкам, поставленным абонентами Мегафона при ответе на второй вопрос.

# Выводы по второму вопросу опроса

&emsp;Не забываем, что все дальнейшие рассуждения по поводу деления параметров на хорошие и плохие по каким либо квантилям, относятся только к значениям параметров для абонентов, поставивших оценку по первому вопросу меньше 9. А значит максимальное значение параметров при анализе ответов на второй вопрос не является абсолютно максимальным для исходных данных.

In [ ]:
good_grade_clean_df = good_grade_df.copy()
q_hi_list = [good_grade_clean_df[col].quantile(0.97)  for col in good_grade_clean_df.columns[4:]]
for i, col in enumerate(good_grade_clean_df.columns[4:]):
    good_grade_clean_df = good_grade_clean_df[good_grade_clean_df[col] < q_hi_list[i]]

In [ ]:
for i, col in enumerate(good_grade_clean_df.columns[3:]):
    if i in [3, 5, 7]:
        proc = round((good_grade_clean_df[good_grade_clean_df[col] > 0][col].min() * 100 / clean_df[clean_df[col] > 0][col].min()), 2)
        print(f'Для параметра {col} минимальное значение при анализе первого вопроса\nсоставляет {proc}% от минимального значения при анализе второго вопроса')
        print(100 * '-')
    else:
        proc = round((clean_df[clean_df[col] > 0][col].max() * 100 / good_grade_clean_df[good_grade_clean_df[col] > 0][col].max()), 2)
        print(f'Для параметра {col} максимальное значение при анализе второго вопроса\nсоставляет {proc}% от максимального значения при анализе первого вопроса')
        print(100 * '-')

Рассмотрим каждый тех. параметр связи и посмотрим, влияет ли он на оценку абонентов при ответе на второй вопрос опроса.

**Примечание**: чтобы повнимательнее посмотреть на какой либо отдельный параметр, надо запустить функцию sf.bootstrep с параметром columns_idxs = [<индекс столбца нужного параметра>] и hypotheses_list = [<0 если лучше меньшее значение параметра и 1 если большее>]

Также для удобства используем функцию sf.p_value_graph, которая для указанного параметра строит графики p-value аналогично её использованию при анализе первого вопроса. Только для анализа оценок по второму вопросу она строит для выбранного параметра 6 графиков, каждый из которых соответствует делению на контрольную и тестовую группы по определённой ответу. Для каждого такого деления идёт подделение на указанные группы еще и по некоему порогу значений исследуемого параметра, выше которого параметр будем считать хорошим, а ниже - плохим. Это деление идёт по квантилям значений параметра, начиная с 95%-ного квантиля и заканчивая 5%-ным квантилем. Для каждого такого порога считается p-value и строится соответствующий график. Таким образом в каждом случае разделения оценок на плохие и хорошие, для каждого порога параметра высчитывается p-value и его значения откладываются по оси y, а по оси x - квантили, по которым делим параметр на плохой и хороший.

In [ ]:
sf.p_value_graph(clean_df,
                      target_feature_idx = 2,
                      column_idx = 3,
                      hypotheses_type = 1,
                      n = -1,
                      N_TRIAL = 3000,
                      func = np.mean)

## 1. Параметр Total Traffic(MB)

### а) ответ "Недозвоны, обрывы при звонках"

&emsp;Очень интересный результат.

&emsp;Ни по методу "by_func", ни по методу "prop" никакой разницы между контрольной и тестовой группой нет.

&emsp;А вот при анализе графика p-value, где мы более детально разбиваем параметр на интервалы, мы видим, что при траффике, равным 75%-ному и 45%-ному квантилю, доля абонентов с хорошим траффиком (больше этого значения) в тестовой группе статистически больше с уровнем доверия 95%, чем в контрольной.

&emsp;**Я вижу такое объяснение. Когда мы делим на контрольную и тестовую группу по высокому значению параметра Total Traffic(MB), мы получаем в тестовой группе тех, кто использует большой объем мобильного интернета - а это скорее те, кто активно смотрит видео и использует какие то другие сервисы, потребляющие большие объемы траффика. А вот при более низком пороге деления параметра на хороший и плохой мы получаем в контрольной группе больший процент абонентов, использующих меньший объем траффика - а это те, кто использует мобильный интернет для сёрфа в соц сетях и !!самое главное!! звонит через мессенджеры, то есть не через мобильную связь, а используя мобильный интернет. И вот для таких абонентов уже появляется разница по ответу "Недозвоны, обрывы при звонках".**

&emsp;**О том же говорит и вид графика. При высоких уровнях траффика - более 75%-ного квантиля - p-value близко к 0.5. Затем резко снижается и при меньших уровнях траффика держится близко к уровню значимости 0.05, пару раз даже пересекая его и показывая разницу между группами.**

### б) ответ "Время ожидания гудков при звонке"

&emsp;Ни по методу "by_func", ни по методу "prop" никакой разницы между контрольной и тестовой группой нет.

&emsp;**Параметр Total Traffic(MB) не влияет на ответ абонентов "Время ожидания гудков при звонке". Никакой динамики в поведении графика p-value не наблюдается.**

### в) ответ "Плохое качество связи в зданиях"

&emsp;Ни по методу "by_func", ни по методу "prop" никакой разницы между контрольной и тестовой группой нет.

&emsp;**Параметр Total Traffic(MB) не влияет на ответ абонентов "Плохое качество связи в зданиях". Но наблюдается динамика смещения значения p-value ы сторону бОльших значений при уменьшении порга деления траффика на хороший и плохой. И при траффике менее 10%-ного квантиля включительно, график p-value пересекает линию значимости 0.95, что говорит о том, что есть основания отвергнуть нулевую гипотезу в пользу гипотезы, обратной альтернативной. То есть при траффике менее 10%-ного квантиля включительно доля абонентов с плохим траффиком выше в тестовой группе. То есть такой "хороший" траффик на уровне 10%-ного квантиля и ниже недостаточен для абонентов и они начинают жаловаться на плохое качество связи в зданиях.**

### г) ответ "Медленный мобильный Интернет"

&emsp;Ни по методу "by_func", ни по методу "prop" никакой разницы между контрольной и тестовой группой нет.

&emsp;**Но при этом при анализе графика p-value мы видим тенденцию к смещению линии разницы долей абонентов с хорошим параметром влево в сторону отрицательных значений по мере уменьшения порога деления значений параметра на хороший и плохой, то есть в сторону гипотезы, обратной альтернативной. Или проще говоря при снижении порога деления параметра на хороший и плохой доля абонентов с хорошим параметром в контрольной группе (то есть выбравшей ответ "Медленный мобильный интернет") увеличивается. Это по моему мнению объясняется тем, что при низких значениях порога деления параметра на хороший и плохой в контрольную группу попадает всё больше абонентов, для которых такой "хороший" параметр не является достаточным, поэтому, несмотря на то, что в тестовой группе абонентов с таким "хорошим" параметром становится больше, чем в контрольной - жалоб на медленный мобильный интернет поступает больше.**

&emsp;**При уровне доверия 95% и делении параметра на хороший и плохой по значению параметра от 5%-ного до 55%-ного квантиля, верна гипотеза, обратная альтернативной. Доля абонентов при этом пороге с хорошим параметром больше у выбравших ответ  "Медленный мобильный интернет". При высоких значениях параметра, он устраимвает всех - и тех, кто использует мобильный интернет для видео, и тех, кто в основном серфит в соц сетях и использует интернет-звонки. Поэтому статистической разницы нет. А вот с 55%-ного квантиля траффик становится неудовлетворительным для абонентов. Мы понимаем, что низкий траффик является следствием чего-то и если в зоне графика выше 55%-ного квантиля уровень траффика определяется потребностями абонента, то в зоне менее 55%-ного квантиля низкий траффик уже объясняется техническими ограничениями и абоненты хотели бы иметь его большие значения, но медленный мобильный интернет не даёт им их получить. Здесь и абоненты, использующие интернет для видео, и те, кто использует его для серфинга в соц сетях. Опять же, если бы мы изначально разделили абонентов на такие группы, задав им напрямую вопрос о целях использования интернета, мы бы получили более точную картину.**

### д) ответ "Медленная загрузка видео"

&emsp;Ни по методу "by_func", ни по методу "prop" никакой разницы между контрольной и тестовой группой нет.

&emsp;**При анализе графика p-value подтверждаются наши выводы. В зоне от 20%-ного квантиля до 80%-ного квантиля график очень близок к линии уровня значимости для альтернативной гипотезы, так как именно скорость загрузки видео касается в первую очередь абонентов, использующих интернет именно для видео и они недовольны уже "хорошим" параметром на уровне 80%-ного квантиля. А вот при траффике ниже 20%-ного квантиля на первый план выходят абоненты, использующие мобильный интернет для соц сетей и интернет-звонков. Им скорость загрузки видео не так важна и так как их доля при низком траффике увеличивается, то и за счет них разница между контрольной и тестовой группой по выбравшим ответ "Медленная загрузка видео" нивелируется.**

### е) ответы "Затрудняюсь ответить" и "Свой вариант"

&emsp;Ни по методу "by_func", ни по методу "prop" никакой разницы между контрольной и тестовой группой нет.

&emsp;**Параметр Total Traffic(MB) не влияет на ответы абонентов  "Затрудняюсь ответить" и "Свой вариант".**

In [ ]:
sf.p_value_graph(clean_df,
                      target_feature_idx = 2,
                      column_idx = 4,
                      hypotheses_type = 1,
                      n = -1,
                      N_TRIAL = 3000,
                      func = np.mean)

## 2. Параметр Downlink Throughput(Kbps)

### а) ответ "Недозвоны, обрывы при звонках"

&emsp;Ни по методу "by_func", ни по методу "prop" никакой разницы между контрольной и тестовой группой нет.

&emsp;**Параметр Downlink Throughput(Kbps) не влияет на ответ абонентов "Недозвоны, обрывы при звонках". Но при этом при анализе графика p-value видим, что он очень близок к уровню значимости 0.95 в диапазоне значений средней скорости "к абоненту" от 70%-ного до 85%-ного квантиля. Почему то в этом диапазоне несмотря на хорошую скорость склонность абонентов пожаловаться на недозвоны и обрывы при звонках практически приближается к статистически значимой.**

### б) ответ "Время ожидания гудков при звонке"

&emsp;Ни по методу "by_func", ни по методу "prop" никакой разницы между контрольной и тестовой группой нет.

&emsp;**Параметр Downlink Throughput(Kbps) не влияет на ответ абонентов "Время ожидания гудков при звонке".**

### в) ответ "Плохое качество связи в зданиях"

&emsp;Ни по методу "by_func", ни по методу "prop" никакой разницы между контрольной и тестовой группой нет.

&emsp;**Параметр Downlink Throughput(Kbps) не влияет на ответ абонентов "Плохое качество связи в зданиях"**

### г) ответ "Медленный мобильный Интернет"

&emsp;По методу "by_func" разница между контрольной и тестовой группой есть.

&emsp;По методу "prop" мы видим следующую картину. Разница появляется при значении порога деления параметра на хороший и плохой уже по 80%-ному квантилю и ниже. Но при значении порога деления параметра на хороший и плохой на уровне 25%-ного и 30%-ного квантиля разница пропадает. О чем это говорит?

&emsp;**Параметр Downlink Throughput(Kbps) влияет с уровнем доверия 95% на ответ абонентов "Медленный мобильный Интернет" при значении порога деления параметра на хороший и плохой менее 80%-ного квантиля включительно. При этом, опираясь на вышеизложенные выводы мы можем сказать, что эти недовольные - это абоненты, использующие интернет в основном для видео и других требовательных к скорости интернета приложений. Им уже не нравится скорость ниже 80% квантиля. А вот при 25%-ном и 30%-ном квантиле в тестовую группу уже начинают попадать абоненты, исползующие интернет в основном для соцсетей и интернет-звонков, поэтому их такая скорость устраивает и разница начинает сокращаться (p-value растёт), доходя до тсатистически неразличимой.**

### д) ответ "Медленная загрузка видео"

&emsp;По методу "by_func" разница между контрольной и тестовой группой есть.

&emsp;По методу "prop" разница есть уже при делении на контрольную и тестовую группы по порогу значения параметра, равному 90%-ному квантилю и p-value сохраняется близким к нулю для всех порогов при делении на группы.

&emsp;**Параметр Downlink Throughput(Kbps) сильно влияет с уровнем доверия 95% на ответ абонентов "Медленная загрузка видео" при любом делении на контрольную и тестовую группу по порогам параметра от 90%-ного квантиля вплоть до медианного. Это говорит о том, что абонентам, использующим интернет для видео недостаточно даже максимальной скорости, значение которой представлено в переданных нам данных, что обусловлено видимо спецификой вдеоконтента. Всё больше видео появляется в высоком разрешении вплоть до 4K и выше, а "Мегафон" зачастую используют и для просмотра видео через смарт-тв на телевизоре, который может такую картинку реально качественно показать. Поэтому именно для видео скорости могут быть и недостаточны, так как разрешение контента и могущих этот контент воспроизвести устройств растёт прямо на глазах. Повышение скорости передачи данных должно быть ориентировано именно на таких абонентов. Если их число будет расти, то придётся модернизировать оборудование.**

### е) ответы "Затрудняюсь ответить" и "Свой вариант"

&emsp;Ни по методу "by_func", ни по методу "prop" никакой разницы между контрольной и тестовой группой нет.

&emsp;**Параметр Downlink Throughput(Kbps) не влияет на ответы абонентов  "Затрудняюсь ответить" и "Свой вариант". При этом по графику p-value видно, что при высоких значениях средней скорости "к абоненту" график очень близок к линии значимости 0.95, что говорит о том, что абоненты ближе к тому, чтобы ответить "Затрудняюсь ответить" и "Свой вариант". Это и понятно - ведь при высоких скоростях меньше поводов на что-то жаловаться.**

In [ ]:
sf.p_value_graph(clean_df,
                      target_feature_idx = 2,
                      column_idx = 5,
                      hypotheses_type = 1,
                      n = -1,
                      N_TRIAL = 3000,
                      func = np.mean)

## 3. Параметр Uplink Throughput(Kbps)

### а) ответ "Недозвоны, обрывы при звонках"

&emsp;По методу "by_func" разница между контрольной и тестовой группой есть, причем хуже значения параметра в тестовой группе.

&emsp;По методу "prop" разница между контрольной и тестовой группой есть при порге деления значений параметра на хороший и плохой по 70%-квантилю и выше.

&emsp;**Мы четко можем утверждать с 95%-ным уровнем доверия, что чем выше значение параметра мы считаем хорошим, тем больше разница между тестовой и контрольной группой, причем больше жалуются на "Недозвоны, обрывы при звонках" абоненты с высоким уровнем параметра Uplink Throughput(Kbps). Также по графику p-value видно, что и при очень плохих показателях Uplink Throughput(Kbps), ниже 15%-ного квантиля он влияет на жалобы абонентов на недозвоны и обрывы при звонках.**

### б) ответ "Время ожидания гудков при звонке"

&emsp;Ни по методу "by_func", ни по методу "prop" никакой разницы между контрольной и тестовой группой нет.

&emsp;**Параметр Uplink Throughput(Kbps) не влияет на ответ абонентов "Время ожидания гудков при звонке". При этом график наиболее близок к 95%-ному уровню значимости при высоких значениях параметра Uplink Throughput(Kbps) в диапазоне от 80%-ного до 90%-ного квантиля включительно.**

### в) ответ "Плохое качество связи в зданиях"

&emsp;Ни по методу "by_func", ни по методу "prop" никакой разницы между контрольной и тестовой группой нет.

&emsp;**Параметр Uplink Throughput(Kbps) не влияет на ответ абонентов "Плохое качество связи в зданиях". При этом судя по графику p-value при более низких значениях параметра абоненты могут быть более склонны пожаловаться на плохое качество связи в зданиях**

### г) ответ "Медленный мобильный Интернет"

&emsp;По методу "by_func" разница между контрольной и тестовой группой есть.

&emsp;По методу "prop" разница между контрольной и тестовой группой есть при значении порога деления параметра на хороший и плохой по квантилю 80% и выше включительно.

&emsp;**При анализе графика p-value видно, что параметр Uplink Throughput(Kbps) влияет на ответ абонентов "Медленный мобильный Интернет" только если делить абонентов на тестовую и контрольную группы по значению параметра от 75%-ного до 90%-ного квантиля включительно либо в диапазоне от 20%-ного до 35%-ного квантиля включительно. Очень странное поведение графика даёт повод техническим специалистам "Мегафон" проверить, почему определённые уровни параметра Uplink Throughput(Kbps) влияют на скорость интернета. С другой стороны эти два диапазона значений параметра, при которых прослеживается статистическая разница между контрольной и тестовой группой, могут быть вызваны как раз таки тем, что один из диапазонов относиться к абонетам, использующим мобильный интернет для видео, а другой - к тем, кто использует мобильный интернет для соц сетей и интернет-звонков. Можно было бы это определить точнее, если бы ввести дополнительный вопрос в опрос клиентов, о котором я писал в выводах в первой части работы.**

### д) ответ "Медленная загрузка видео"

&emsp;По методу "by_func" разница между контрольной и тестовой группой есть.

&emsp;По методу "prop" разница между контрольной и тестовой группой есть при значениях порога деления параметра на хороший и плохой по квантилю 70% и ниже.

&emsp;**Анализ графика p-value показывает, что параметр Uplink Throughput(Kbps) влияет на ответ абонентов "Медленная загрузка видео" только если делить абонентов на тестовую и контрольную группы по значению параметра, равному 75%-ному квантилю или ниже.  Это подтверждает нашу теорию о том, что абоненты делятся на две группы по цели использования интернета. Данный параметр наиболее важен именно для абонентов группы, использующей интернет для видео. Поэтому и график показывает стабильное поведение. При уровнях Uplink Throughput(Kbps) выше 75%-ного квантиля абоненты не чувствительны к скорости загрузки видео, а вот при более низких относительно этого порога значениях появляются жалобы на медленную загрузку видео.**

### е) ответы "Затрудняюсь ответить" и "Свой вариант"

&emsp;Ни по методу "by_func", ни по методу "prop" никакой разницы между контрольной и тестовой группой нет.

&emsp;**Параметр Uplink Throughput(Kbps) не влияет на ответы абонентов  "Затрудняюсь ответить" и "Свой вариант"**

In [ ]:
sf.p_value_graph(clean_df,
                      target_feature_idx = 2,
                      column_idx = 6,
                      hypotheses_type = 0,
                      n = -1,
                      N_TRIAL = 3000,
                      func = np.mean)

## 4. Параметр Downlink TCP Retransmission Rate(%)

&emsp;Не забываем, что при анализе графиков p-value для этого параметра рассматриваем график слева направо, так как здесь лучше меньшие значения параметра.

### а) ответ "Недозвоны, обрывы при звонках"

&emsp;Ни по методу "by_func", ни по методу "prop" никакой разницы между контрольной и тестовой группой нет.

&emsp;**Параметр Downlink TCP Retransmission Rate(%) практически не влияет на ответ абонентов "Недозвоны, обрывы при звонках", кроме одного случая. При уровне параметра Downlink TCP Retransmission Rate(%), равного 30%-ному квантилю, доля абонентов с такими значениями параметра в тестовой группе с уровнем доверия 95% почти статистически отличается в большую сторону от доли абонентов с этими же значениями параметра в контрольной группе. У графика p-value есть ярко выраженный экстремум. Только при частоте переотправок пакетов к абоненту на уровне 30%-ного квантиля абоненты жалуются на недозвоны и обрывы при звонках. Повод разобраться в этом специалистам "Мегафона".**

### б) ответ "Время ожидания гудков при звонке"

&emsp;По методу "by_func" разницы между контрольной и тестовой группой нет.

&emsp;По методу "prop" разница между контрольной и тестовой группой есть только при значении параметра, равном 30%-ному квантилю.

&emsp;**Опять же, как и в случае с недозвонами есть узкий диапазон значений параметра Downlink TCP Retransmission Rate(%) от 30%-ного до 35%-ного квантиля включительно, при котором абоненты жалуются на время ожидания гудков при звонке.**

### в) ответ "Плохое качество связи в зданиях"

&emsp;По методу "by_func" присутствует разница между контрольной и тестовой группой, если смотреть по вхождению нуля в доверительный интервал разниц средних групп. При этом ноль находится практически на границе интервала, балансируя на грани статистической неразличимости.

&emsp;По методу "prop" разницы между контрольной и тестовой группой нет.

&emsp;**А вот анализ графика p-value даёт интересную картину. Разница появляется при значении параметра Downlink TCP Retransmission Rate(%) на уровне 15%-ного квантиля, а также при значении параметра в диапазоне от 70%-ного до 90%-ного квантиля включительно. То есть опятьже видно, что параметр спецефический. Есть какие то значения вблизи 15%-ного квантиля, при которых при уровне доверия 95% можно говорить о зависимости ответа абонентов "Плохое качество связи в зданиях" от параметра, а также при достаточно плохих значениях параметра в вышеназванном диапазоне также абоненты чувствуют это в виде плохого качества связи в зданиях.**

### г) ответ "Медленный мобильный Интернет"

&emsp;По методу "by_func" присутствует разница между контрольной и тестовой группой, если смотреть по вхождению нуля в доверительный интервал разниц средних групп. При этом ноль находится практически на границе интервала, балансируя на грани статистической неразличимости.

&emsp;По методу "prop" разница появляется при значении порога деления параметра на хороший и плохой, равного 40%-ному квантилю и выше.

&emsp;**Анализ графика p-value даёт нам опять же интересную картину. При очень высоких и очень низких значениях параметра, абоненты не чувствуют разницы по скорости мобильного интернета. А вот при средних значениях Downlink TCP Retransmission Rate(%) в диапазоне от 40%-ного до 70%-ного квантиля включительно при уровне доверия 95% можно сказать, что значения параметра влияют на ответ абонентов "Медленный мобильный Интернет".**

### д) ответ "Медленная загрузка видео"

&emsp;Ни по методу "by_func", ни по методу "prop" никакой разницы между контрольной и тестовой группой нет.

&emsp;**Параметр Downlink TCP Retransmission Rate(%) не влияет на ответ абонентов "Медленная загрузка видео"**

### е) ответы "Затрудняюсь ответить" и "Свой вариант"

&emsp;По методу "by_func" разницы между контрольной и тестовой группой нет.

&emsp;Как ни странно при методе "prop" и значении порога деления параметра на плохой и хороший, равного 40%-ному квантилю, появляется статистическая разница между контрольной и тестовой группой. Причем верна гипотеза, обратная альтернативной.

&emsp;**Анализ графика p-value показывает, что параметр Downlink TCP Retransmission Rate(%) с уровнем доверия 95% влияет на ответ абонентов "Затрудняюсь ответить" и "Свой вариант" при некоторых значении порога деления параметра на плохой и хороший. Но статистическая разница появляется и при низких, и при высоких, и при средних значениях параметра. Причём доля хороших значений параметра у абонентов, ответивших так (а точнее не ответивших ничего) выше, чем доля абонентов с хорошим параметром, выбравших какой либо вариант ответа. То есть несмотря на то, что параметр хороший, абоненты предпочли не отвечать на второй вопрос как то конкретно и затруднились ответить. Скорее всего это происходит потому, что данный параметр сложно интерпретируем для абонентов. Он зависим от остальных параметров связи и скорее всего такая статистическая разница для абонентов, выбравших данный вариант ответа отчасти случайна.**

In [ ]:
sf.p_value_graph(clean_df,
                      target_feature_idx = 2,
                      column_idx = 7,
                      hypotheses_type = 1,
                      n = -1,
                      N_TRIAL = 3000,
                      func = np.mean)

## 5. Параметр Video Streaming Download Throughput(Kbps)

### а) ответ "Недозвоны, обрывы при звонках"

&emsp;По методу "by_func" разницы между контрольной и тестовой группой нет.

&emsp;По методу "prop" разница появляется при значении порога деления параметра на хороший и плохой, равного 60%-ному и 70%-ному квантилю, причем срабаьтывает гипотеза, обратная альтернативной.

&emsp;**Анализ графика p-value показывает, что параметр Video Streaming Download Throughput(Kbps) влияет на ответ абонентов "Недозвоны, обрывы при звонках" при значении порога деления параметра на плохой и хороший в диапазоне от 55%-ного до 70%-ного квантиля включительно, причём доля абонентов с хорошим параметром в контрольной группе (то есть жалующихся на недозвоны) выше, чем в тестовой. То есть при лучшей скорости загрузки потокового видео при значениях параметра чуть выше медианных есть статистически достоверное с уровнем доверия 95% ухудшение стабильности интернет-звонков.**

### б) ответ "Время ожидания гудков при звонке"

&emsp;Ни по методу "by_func", ни по методу "prop" никакой разницы между контрольной и тестовой группой нет.

&emsp;**Параметр Video Streaming Download Throughput(Kbps) не влияет на ответ абонентов "Время ожидания гудков при звонке"**

### в) ответ "Плохое качество связи в зданиях"

&emsp;Ни по методу "by_func", ни по методу "prop" никакой разницы между контрольной и тестовой группой нет.

&emsp;**Параметр Video Streaming Download Throughput(Kbps) не влияет на ответ абонентов "Плохое качество связи в зданиях"**

### г) ответ "Медленный мобильный Интернет"

&emsp;И по методу "by_func", и по методу "prop" есть большая разница между контрольной и тестовой группой.

&emsp;**Параметр Video Streaming Download Throughput(Kbps) очень сильно влияет на ответ абонентов "Медленный мобильный Интернет". Причём даже если считать хорошим параметр со значением выше 95%-ного квантиля. То есть скорость мобильного интернета плохо удовлетворяет даже тех абонентов, значение скорости загрузки потокового видео у которых близко к максимуму. В принципе причины мы вывели ранее - рост количества видеоконтента с высоким разрешением.**

### д) ответ "Медленная загрузка видео"

&emsp;И по методу "by_func", и по методу "prop" есть большая разница между контрольной и тестовой группой.

&emsp;**Анализ графмка p-value показывает, что параметр Video Streaming Download Throughput(Kbps) очень сильно влияет на ответ абонентов "Медленный мобильный Интернет". При это при очень хороших значениях выше 90%-ного квантиля абоненты удовлетворены скоростью загрузки видео, а при низких значениях ниже 15%-ного квантиля, начинают влиять на ответ абонениы, использующие интернет для соц сетей и интернет-звонков. Их при этом уровне параметра становится больше и им не важна скорость загрузки видео, поэтому за счет них пропадает зависимость ответа от значения параметра.**

### е) ответы "Затрудняюсь ответить" и "Свой вариант"

&emsp;По методу "by_func" разницы между контрольной и тестовой группой нет.

&emsp;По методу "prop" есть разница между контрольной и тестовой группой, причём верна обратная альтернативной гипотеза.

&emsp;**Параметр Video Streaming Download Throughput(Kbps) с уровнем доверия 95% при значении параметра выше 25%-ного квантиля влияет на ответ абонентов "Затрудняюсь ответить" и "Свой вариант". Причём у контрольной группы (то есть у тех, кто так ответил), доля абонентов с хорошим параметром выше. Видимо если скорость загрузки потокового видео достаточно высока, то большАя доля абонентов более менее удовлетворена и остальными параметрами, поэтому среди ответов не находит подходящий и затрудняется ответить или не отвечает вовсе.**

In [ ]:
sf.p_value_graph(clean_df,
                      target_feature_idx = 2,
                      column_idx = 8,
                      hypotheses_type = 0,
                      n = -1,
                      N_TRIAL = 3000,
                      func = np.mean)

## 6. Параметр Video Streaming xKB Start Delay(ms)

&emsp;Не забываем, что при анализе графиков p-value для этого параметра рассматриваем график слева направо, так как здесь лучше меньшие значения параметра.

### а) ответ "Недозвоны, обрывы при звонках"

&emsp;Ни по методу "by_func", ни по методу "prop" никакой разницы между контрольной и тестовой группой нет.

&emsp;**Параметр Video Streaming xKB Start Delay(ms) не влияет на ответ абонентов "Недозвоны, обрывы при звонках"**

### б) ответ "Время ожидания гудков при звонке"

&emsp;Ни по методу "by_func", ни по методу "prop" никакой разницы между контрольной и тестовой группой нет.

&emsp;**Параметр Video Streaming xKB Start Delay(ms) не влияет на ответ абонентов "Время ожидания гудков при звонке"**

### в) ответ "Плохое качество связи в зданиях"

&emsp;Ни по методу "by_func", ни по методу "prop" никакой разницы между контрольной и тестовой группой нет.

&emsp;**Параметр Video Streaming xKB Start Delay(ms) не влияет на ответ абонентов "Плохое качество связи в зданиях"**

### г) ответ "Медленный мобильный Интернет"

&emsp;И по методу "by_func", и по методу "prop" есть большая разница между контрольной и тестовой группой.

&emsp;**Параметр Video Streaming xKB Start Delay(ms) очень сильно влияет на ответ абонентов "Медленный мобильный Интернет". То есть при любом делении параметра на хороший и плохой с уровнем доверия 95% есть разница между контрольной и тестовой группой. Чем меньше задержка старта воспроизведения видео, тем меньше жалоб на скорость мобильного интернета.**

### д) ответ "Медленная загрузка видео"

&emsp;По методу "by_func" разницы между контрольной и тестовой группой нет.

&emsp;По методу "prop" разница между контрольной и тестовой группой есть.

&emsp;**Анализируя график p-value, видим, что параметр Video Streaming xKB Start Delay(ms) влияет на ответ абонетов "Медленная загрузка видео" при значении порога деления параметра на хороший и плохой вплоть до 75%-ного квантиля за исключением узкого диапазона от 55%-ного до 60%-ного квантиля. При высоких (то есть плохих) значениях задержки старта воспроизведения видео ответ перестает зависеть от параметра. Это обусловлено видимо тем, что при таких задержках эта зона графика находится под влиянием абонентов, использующих интернет для соц сетей и интернет-звонков и им не важен параметр, связанный с видео.**

### е) ответы "Затрудняюсь ответить" и "Свой вариант"

&emsp;Ни по методу "by_func", ни по методу "prop" никакой разницы между контрольной и тестовой группой нет.

&emsp;**При анализе графика p-value видим, что параметр Video Streaming xKB Start Delay(ms) не влияет на ответы абонентов "Затрудняюсь ответить" и "Свой вариант", кроме пары значений в районе 25%-ного и 85%-ного квантиля. Возможно это связано с делением абонентов на использующих интернет для видео и для соц сетей. Возможно в этих точках превалирует та или иная группа и совместно дают такой результат.**

In [ ]:
sf.p_value_graph(clean_df,
                      target_feature_idx = 2,
                      column_idx = 9,
                      hypotheses_type = 1,
                      n = -1,
                      N_TRIAL = 3000,
                      func = np.mean)

## 7. Параметр Web Page Download Throughput(Kbps)

### а) ответ "Недозвоны, обрывы при звонках"

&emsp;Ни по методу "by_func", ни по методу "prop" никакой разницы между контрольной и тестовой группой нет.

&emsp;**Параметр Web Page Download Throughput(Kbps) не влияет на ответ абонентов "Недозвоны, обрывы при звонках"**

### б) ответ "Время ожидания гудков при звонке"

&emsp;Ни по методу "by_func", ни по методу "prop" никакой разницы между контрольной и тестовой группой нет.

&emsp;**А вот при анализе графика p-value мы видим, что график в диапазоне низких значений скорости загрузки web-страниц через браузер, вплоть до 30%-ного квантиля включительно, лежит выше уровня значимости 95%. То есть при таких уровнях порога деления параметра на хороший и плохой жалуются на время ожидания гудков абоненты с хорошим относительно порога параметром. То есть такой "хороший" параметр для них недостаточен и каким то образом связан с ожиданием гудков при звонке.**

### в) ответ "Плохое качество связи в зданиях"

&emsp;Ни по методу "by_func", ни по методу "prop" никакой разницы между контрольной и тестовой группой нет.

&emsp;**То же самое. При анализе графика p-value мы видим, что график в диапазоне низких значений скорости загрузки web-страниц через браузер, вплоть до 30%-ного квантиля включительно, лежит выше уровня значимости 95%. То есть при таких уровнях порога деления параметра на хороший и плохой жалуются на плохое качество связи в зданиях абоненты с хорошим относительно порога параметром. То есть такой "хороший" параметр для них недостаточен и каким то образом связан с плохим качеством связи в зданиях.**

### г) ответ "Медленный мобильный Интернет"

&emsp;И по методу "by_func", и по методу "prop" есть большая разница между контрольной и тестовой группой.

&emsp;**Параметр Web Page Download Throughput(Kbps) сильно влияет на ответ абонентов "Медленный мобильный Интернет". То есть при любом делении параметра на хороший и плохой с уровнем доверия 95% есть разница между контрольной и тестовой группой. Чем лучше скорость загрузки web-страниц через браузер, тем меньше жалоб на скорость мобильного интернета.**

### д) ответ "Медленная загрузка видео"

&emsp;И по методу "by_func", и по методу "prop" есть большая разница между контрольной и тестовой группой.

&emsp;**При анализе графика p-value мы видим, что параметр при уровне доверия 95% влияет на ответ абонентов "Медленная загрузка видео" только в диапазоне значений от 30%-ного квантиля до 90%-ного квантиля включительно. При низких значениях скорости загрузки веб страниц до 30%-ного квантиля абоненты не чувствуют разницы в скорости загрузки видео. Здесь видимо больше влияние группы абонентов, использующих интернет для видео. Им не важна скорость загрузки веб-страниц. А вот при высоких значениях параметра выше 90%-ного квантиля сказывается влияние абонентов, использующих интернет для соц сетей и тому подобного. Для них важен параметр скорости загрузки веб-страниц и при очень высоком его значении они уже не ощущают его улучшения. Он и так уже достаточен для них.**

### е) ответы "Затрудняюсь ответить" и "Свой вариант"

&emsp;По методу "by_func" присутствует разница между контрольной и тестовой группой, если смотреть по вхождению нуля в доверительный интервал разниц средних групп. При этом ноль находится практически на границе интервала, балансируя на грани статистической неразличимости.

&emsp;По методу "prop" есть разница между контрольной и тестовой группой, кроме случая деления параметра на хороший и плохой по 90%-ному квантилю. Причём верна обратная альтернативной гипотеза.

&emsp;**При анализе графика p-value видим, что параметр Web Page Download Throughput(Kbps) влияет на ответ абонентов "Затрудняюсь ответить" и "Свой вариант" в диапазоне значений параметра от 40%-ного до 85%-ного квантиля включительно. Причём у контрольной группы (то есть у тех, кто так ответил), доля абонентов с хорошим параметром выше. Видимо если скорость загрузки web-страниц через браузер достаточно высока, то большАя доля абонентов более менее удовлетворена и остальными параметрами, поэтому среди ответов не находит подходящий и затрудняется ответить или не отвечает вовсе.**

In [ ]:
sf.p_value_graph(clean_df,
                      target_feature_idx = 2,
                      column_idx = 10,
                      hypotheses_type = 0,
                      n = -1,
                      N_TRIAL = 3000,
                      func = np.mean)

## 8. Параметр Web Average TCP RTT(ms)

&emsp;Не забываем, что при анализе графиков p-value для этого параметра рассматриваем график слева направо, так как здесь лучше меньшие значения параметра.

### а) ответ "Недозвоны, обрывы при звонках"

&emsp;Ни по методу "by_func", ни по методу "prop" никакой разницы между контрольной и тестовой группой нет.

&emsp;**Параметр Web Average TCP RTT(ms) не влияет на ответ абонентов "Недозвоны, обрывы при звонках"**

### б) ответ "Время ожидания гудков при звонке"

&emsp;Ни по методу "by_func", ни по методу "prop" никакой разницы между контрольной и тестовой группой нет.

&emsp;**Параметр Web Average TCP RTT(ms) не влияет на ответ абонентов "Время ожидания гудков при звонке"**

### в) ответ "Плохое качество связи в зданиях"

&emsp;По методу "by_func" разницы между контрольной и тестовой группой нет.

&emsp;По методу "prop" разница между контрольной и тестовой группой есть при значениях порога деления параметра на хороший и плохой по квантилю 20% и 50%.

&emsp;**При определённых уровнях пинга при уровне доверия 95% появляется статистическая разница между контрольной и тестовой группой. При этом верна гипотеза, обратная альтернативной, то есть доля абонентов с хорошим параметром Web Average TCP RTT(ms) больше у абонентов, ответивших  "Плохое качество связи в зданиях". Значит параметр Web Average TCP RTT(ms) влияет на ответ абонентов "Плохое качество связи в зданиях" при значении этого параметра около 20%-ного и более 50%-ного квантиля. При этом наблюдается обратная зависимость - больше жалуются на плохое качество связи в зданиях абоненты с хорошим показателем пинга Web Average TCP RTT(ms), если считать хорошим значения параметра около 20%-ного и более 50%-ного квантиля.**

### г) ответ "Медленный мобильный Интернет"

&emsp;И по методу "by_func", и по методу "prop" есть разница между контрольной и тестовой группой.

&emsp;**Параметр Web Average TCP RTT(ms) влияет на ответ абонентов "Медленный мобильный Интернет". Но при анализе графика p-value видно, что при значении порога параметра выше 75%-ного квантиля, то есть при плохом пинге, разница между контрольной и тестовой группой нивелируется и пинг перестает влиять на ответ абонентов "Медленный мобильный Интернет". Здесь видимо сказывается влияние группы абонентов, использующих интернет для видео. У них скорость интернета видимо достаточная, а пинг при просмотре веб-страниц им не важен.**

### д) ответ "Медленная загрузка видео"

&emsp;По методу "by_func" присутствует разница между контрольной и тестовой группой, если смотреть по вхождению нуля в доверительный интервал разниц средних групп.

&emsp;По методу "prop" разницы между контрольной и тестовой группой нет только при значениях порога деления параметра на хороший и плохой по квантилю 40%.

&emsp;**При анализе графика p-value видно, что в широком диапазоне значений пинга за небольшими исключениями есть влияние этого параметра на ответ абонентов "Медленная загрузка видео". Скорее всего здесь идёт взаимное влияние групп абонентов, использующих интернет для видео и для соц сетей. Для первых этот параметр не важен и там где график выходит из зоны значимости скорее всего в группах преобладают такие абоненты. Где разница существенна, скорее преобладают вторые.**

### е) ответы "Затрудняюсь ответить" и "Свой вариант"

&emsp;По методу "by_func" разницы между контрольной и тестовой группой нет.

&emsp;По методу "prop" разница между контрольной и тестовой группой есть только при значении порога деления параметра на хороший и плохой, равного 30%-ному квантилю. При этом верна гипотеза, обратная альтернативной - доля абонетов с хорошим параметром при таком пороге выше в контрольной группе.

&emsp;**По большей части разница между контрольной и тестовой группой возникает при низких (то есть хороших) значениях пинга. То есть при хорошем пинге больше вероятность, что абонент удовлетворен в целом качеством остальных параметров и не может пожаловаться на какой то из них.**

# Общий вывод по второму вопросу для бизнеса

&emsp;1. Ответ "Недозвоны, обрывы при звонках" более вероятен при низком уровне траффика у абонентов. (Хотя это скорее косвенный показатель. Ведь низкий траффик может быть обусловлен низким качеством других параметров). В какой то мере в разных диапазонах значений на этот овет влияют параметры Downlink Throughput(Kbps), Uplink Throughput(Kbps), Downlink TCP Retransmission Rate(%) и Video Streaming Download Throughput(Kbps). Остальные параметры на ответ "Недозвоны, обрывы при звонках" не влияют. Стоит обратить внимание на частоту переотправок пакетов "к абоненту". Только при значении этого параметра на уровне 30%-ного квантиля абоненты жалуются на недозвоны и обрывы при звонках. Техническим специалистам "Мегафон" стоит обратить на это внимание - в чем причина. 

&emsp;2. Ответ "Время ожидания гудков при звонке" более вероятен при низкой скорости загрузки веб-страниц через браузер и при высокой средней скорости "от абонента". Также опять же стоит обратить внимание на частоту переотправок пакетов "к абоненту". Примерно при тех же значениях, что и в случае с ответом "Недозвоны, обрывы при звонках" - оклоло 30%-ного квантиля - абоненты жалуются на время ожидания гудков при звонке. Второй повод обратить на эту аномалию внимание специалистам "Мегафон".

&emsp;3. Ответ "Плохое качество связи в зданиях" более вероятен при низком уровне траффика у абонентов, низкой средней скорости "от абонента", низкой скорости загрузки веб-страниц и высоком значении пинга при просмотре web-страниц (больше медианного). Также опять аномальные показатели относительно частоты переотправок пакетов "к абоненту". Вероятность жалоб на плохое качество связи в зданиях выше у абонентов с определенным уровнем этой частоты - около 15%-ного квантиля и в диапазоне от 70%-ного до 90%-ного квантиля включительно.

&emsp;4. Ответ "Медленный мобильный Интернет" и ответ "Медленная загрузка видео" в большинстве своем зависят от технических параметров связи. При этом чётко видно влияние на эти ответы распределения абонентов на группы по целевому использованию интернета. Вывод и рекомендация аналогичны пункту 1 выводов по первой части работы при анализе распределения оценок абонентов при ответе на первый вопрос опроса.

&emsp;5. Ответы "Затрудняюсь ответить" и "Свой вариант" более вероятны при высоких значениях параметров, когда абоненты удовлетворены какими то из них в достаточной мере и затрудняются выбрать соответствующую проблему из списка предложенных.

# Пожелания и рекомендации

&emsp;Поработав с данными, у меня появился некий план действий для улучшения удовлетворенности клиентов "Мегафон"

&emsp;1. Надо разделить абонентов по территориальному признаку по принципу на некие регионы.

&emsp;2. В пределах региона разделить абонентов по принципу одинаковости оборудования связи на близко расположенных вышках.

&emsp;3. Проводить опросы, подобные этому, с добавлением вопроса - "Для чего в основном вы используете мобильный интернет?". И трех вариантов ответа - "Для просмотра видео, в том числе на смарт тв приставках", "Для соц.сетей, мессенджеров и интернет-звонков" и "И для того, и для другого в равной степени".

&emsp;4. Для каждой получившейся группы проводить исследования, подобные этому, используя мои универсальные функции. Несмотря на их громоздкость, они работают очень быстро и способны переварить гораздо бОльшие объемы данных.

&emsp;5. Проводить такие исследования с определённой периодичностью, подгружая новые данные за период и следя за динамикой изменений.

&emsp;**Таким образом для каждой группы вышек с одинаковым оборудованием для каждого региона мы получим данные, согласно которым будет понятно, какие настройки надо произвести или какие элементы оборудования модернизировать или обновить, чтобы максимально удовлетворить клиентов в данном конкретном регионе.**